# Word Representation 简介

## One Hot Representation

In [47]:
import numpy as np
from gensim.corpora import Dictionary
import pprint

将文档进行分词

In [35]:
sent1 = 'this is a foo bar sentence'.split()
sent2 = 'this is another foo bar sentence'.split()
texts = [sent1, sent2]

根据词频为每个单词指定 ID

In [36]:
vocab = Dictionary(texts)
list(vocab.items())

[(0, 'a'),
 (1, 'bar'),
 (2, 'foo'),
 (3, 'is'),
 (4, 'sentence'),
 (5, 'this'),
 (6, 'another')]

将每个单词的ID转换为onehot形式

In [37]:
def to_onehot(dict_len, word_id):
    onehot = np.zeros(dict_len)
    onehot[word_id] = 1
    return onehot
[(word, to_onehot(len(vocab.items()), idx)) for idx, word in vocab.items()]

[('a', array([1., 0., 0., 0., 0., 0., 0.])),
 ('bar', array([0., 1., 0., 0., 0., 0., 0.])),
 ('foo', array([0., 0., 1., 0., 0., 0., 0.])),
 ('is', array([0., 0., 0., 1., 0., 0., 0.])),
 ('sentence', array([0., 0., 0., 0., 1., 0., 0.])),
 ('this', array([0., 0., 0., 0., 0., 1., 0.])),
 ('another', array([0., 0., 0., 0., 0., 0., 1.]))]

** 缺点 **
* one-hot的表示方法是一种稀疏表示方式，当词典非常大时会造成维数灾难，非常高维的向量计算非常困难
* one-hot表示方法表示的两个词的词向量是孤立的，不能从两个词的向量中看出两个词之间的语义关系

## Distributed representation

 * ** word embedding 是什么？ **
    * embedding 是个拓扑学的词。地图就是对于现实地理的embedding，现实的地理地形的信息其实远远超过三维，但是地图通过颜色和等高线等来最大化表现现实的地理信息。
    * 计算机在处理自然语言时，无法直接处理单词，所以需要用固定的维度来最大化表现词的信息，也就是所谓的 word embedding，那么 onehot 也是一种 word embedding 的方式
    
* ** Distributed representation 要解决的问题? **
    * 增加特征信息，onehot 解决了词的表示，但是没能最大化表现词的信息，典型的如，语义近似、单复数、时态信息、类比性质（这个性质非常重要，被用来衡量词向量训练结果的好坏）都被丢弃了
    * 降低维度

* ** word2vec 是什么？ **
    * 一种 word embedding 的方法
    * 谷歌开源的工具，其中实现了 CBOW(Continuous Bagof-Words) 和 Skip-gram 模型，那就意味着还有很多其他的模型
    
* ** word2vec 的基本假设 **
    * 分布式假设(distributional hypothesis)，出现在相同上下文(context)下的词意思应该相近。所有学习word embedding的方法都是在用数学的方法建模词和context之间的关系。例如下面两个例子，京巴和泰迪的语义相近：
        * 这个可爱的 ** 泰迪 ** 舔了我的脸。
        * 这个可爱的 ** 京巴 ** 舔了我的脸。

# Word2Vec 算法简介

## CBOW

![cbow](./CBOW.png)

图中 $\omega _{t-c}$ 就是我们的 $x$，也就是 $word2vec$ 的训练目标，CBOW 的思路是用上下文推测当前单词，例如：** 我 是 华为 的 软件开发 人员 ** 那么我们会得到下面几个训练数据

In [46]:
text = "我 是 华为 的 软件开发 人员"
word_list = text.split()
train_sample = [[(word_list[c-2], word_list[c-1], word_list[c+1], word_list[c+2]), word_list[c]] for c in range(2, len(word_list)-2)]
pprint.pprint(train_sample, width=50)

[[('我', '是', '的', '软件开发'), '华为'],
 [('是', '华为', '软件开发', '人员'), '的']]


然后将词初始化为固定长度的向量，向量的值是随机的

In [49]:
wordembedding = {v:[i]*5 for i, v in enumerate(word_list)}
wordembedding

{'人员': [5, 5, 5, 5, 5],
 '华为': [2, 2, 2, 2, 2],
 '我': [0, 0, 0, 0, 0],
 '是': [1, 1, 1, 1, 1],
 '的': [3, 3, 3, 3, 3],
 '软件开发': [4, 4, 4, 4, 4]}

将训练数据转为初始化的 word vector

In [53]:
train_x, train_y = train_sample[0]
train_x, [wordembedding[x] for x in train_x]
train_y, wordembedding[train_y] 

(('我', '是', '的', '软件开发'),
 [[0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [3, 3, 3, 3, 3], [4, 4, 4, 4, 4]])

('华为', [2, 2, 2, 2, 2])

然后将$x$传入投影层

In [57]:
np.sum([wordembedding[x] for x in train_x], axis=0)

array([8, 8, 8, 8, 8])

到这里就很熟悉了，我们通过样本 $x=[8, 8, 8, 8, 8]$ $y=[2, 2, 2, 2]$ 去训练一个模型。
$ML$算法大体可以理解为
$$ y=wx+b $$
其中 $ y $ 是输出， $ x $ 是输入， $ w $ 和 $ b $ 是模型中的参数。我们向模型传入大量的训练数据，即 $(x, y)$，然后，通过反向传播和梯度下降，不断优化 $w$ 和 $b$，但是在 $word2vec$ 中，我们需要优化的是 $x$ 所以，在迭代过程中，我们需要将 $w$ 和 $b$ 视为固定值，然后通过反向传播和梯度下降去优化 $x$

在 $CBOW$ 中 $x$ 是多个词的简单加和，所以，会将梯度下降的程度平均返回到各个 $word vector$ 中，如 $x$ 由 $[8, 8, 8, 8, 8]$ 优化到 $[7, 7, 7, 7, 7]$，则每个 $word vector$ 都会减小 $0.25$

## Skip-gram

  ![skip-gram](./skip-gram.png)

与 CBOW 的区别
* 输入和输出相反，所以输入只有一个vector，不需要做加和了

## Hierarchical Softmax 与 Negative Sampling

两种提升 $wordembedding$ 的算法，其中 $Negative Sampling$ 性能更好。$Hierarchical Softmax$ 相较于传统的 $Softmax$ 性能已经有大幅提升，时间复杂度由 $O(N)$ 降低到 $O(Log_{2}(N))$

# sentences/paragraphs/documents Representation 简介

* bag of words
    * 有如下缺点：1.没有考虑到单词的顺序，2.忽略了单词的语义信息。因此这种方法对于短文本效果很差，对于长文本效果一般
* LDA
    * 计算出一片文档或者句子的主题分布
* average word vectors
    * 简单的对句子中的所有词向量取平均。是一种简单有效的方法，但缺点也是没有考虑到单词的顺序
* tfidf-weighting word vectors
    * 对句子中的所有词向量根据tfidf权重加权求和，是常用的一种计算sentence embedding的方法，在某些问题上表现很好，相比于简单的对所有词向量求平均，考虑到了tfidf权重，因此句子中更重要的词占得比重就更大。但缺点也是没有考虑到单词的顺序

# Doc2Vec 算法简介
首先，Doc2Vec 能做什么，与 word2vec 一样，Doc2Vec 只能将他见过的 doc 转换为 vector，即，只能转换训练集中出现过的 doc，那么 doc2vec 将不适用于 doc 搜索任务，因为用户新输入的 doc 无法转换成 vector，然后再去已有的数据库中查找相似文档，但是 doc2vec 可以对训练集中的 doc 进行转换，抽取出语义级别的信息，然后根据语义近似的信息对 doc 进行分类，典型的，如，情感分析

## DM (distributed memory)
![DM](./DM.png)
类似于 CBOW。DM模型在训练时，首先将每个文档ID和语料库中的所有词初始化一个K维的向量，然后将文档向量和上下文词的向量输入模型，隐层将这些向量累加（或取均值、或直接拼接起来）得到中间向量，作为输出层softmax的输入。在一个文档的训练过程中，文档ID保持不变，共享着同一个文档向量，相当于在预测单词的概率时，都利用了真个句子的语义。


## DBOW (distributed bag of words)
![DBOW](./DBOW.png)
类似于Skip-gram，DBOW模型的输入是文档的向量，预测的是该文档中随机抽样的词。